<a href="https://colab.research.google.com/github/jwkanggist/EverybodyTensorflow2.0/blob/master/lab20_basic_rnn_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB20 Basic RNN using tf keras RNN

- 간단한 seq to seq RNN을 구현해 볼까?
- Tanh activation
- TWO RNN  CELL → Single state RNN 
- 5 recurrent neuron 
- Each Xt  is 3 x 1 vector
- Each Yt is 5 x 1 vector


In [1]:
# preprocessor parts

from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

tf.__version__

TensorFlow 2.x selected.


'2.1.0-rc1'

In [2]:
  # for Tensorboard use
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-01-18 15:17:50--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.103.164, 52.73.147.107, 35.168.165.30, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.103.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  19.3MB/s    in 0.7s    

2020-01-18 15:17:56 (19.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://6a81be95.ngrok.io


In [4]:
# test sequential data
X0 = np.array([[0,1,2],
              [3,4,5],
              [6,7,8],
              [9,0,1],
              ]).reshape([4,1,3])

X1 = np.array([[9,8,7],
                [6,5,4],
                [3,2,1],
                [0,9,8]]).reshape([4,1,3])


X =  np.concatenate([X0,X1],axis=1)
print("X0.shape = %s" % str(X0.shape))
print("X1.shape = %s" % str(X1.shape))
print("X.shape = %s" % str(X.shape))
X

X0.shape = (4, 1, 3)
X1.shape = (4, 1, 3)
X.shape = (4, 2, 3)


array([[[0, 1, 2],
        [9, 8, 7]],

       [[3, 4, 5],
        [6, 5, 4]],

       [[6, 7, 8],
        [3, 2, 1]],

       [[9, 0, 1],
        [0, 9, 8]]])

In [5]:
model_config = \
{
    'batch_size': None,
    'n_input'   : 3,
    'n_output'  : 5,
    'num_steps' : 2,
    'dtype'     : tf.float32
}

# [batch,step,ch]
max_steps = 2
channels = 3

net_in = tf.keras.layers.Input(shape=(max_steps,channels))
net    = tf.keras.layers.SimpleRNN(units=5,activation='tanh')(net_in)
model  = tf.keras.models.Model(inputs=net_in,outputs=net)


opt_fn = tf.keras.optimizers.Adam(learning_rate=1e-4,
                                beta_1=0.9,
                                beta_2=0.999)

# 'sparse_categorical_crossentropy' is for integer labels
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 3)]            0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 45        
Total params: 45
Trainable params: 45
Non-trainable params: 0
_________________________________________________________________


In [13]:
# prediction 
test_input = X[0,:,:]
pred_y = model.predict(test_input.reshape([1,max_steps,channels]))
print(pred_y)


[[ 0.99998224 -0.9999956   0.99915075 -0.9985691  -0.99782944]]
